Authors of the notebook: Guillermo Rangel (guillermo.breto@thinkbiganalytics.com), Alexey Svyatkovskiy (alexeys@princeton.edu), Ricardo Vasquez (rvasquezsierra@gmail.com)


#Data matching and deduplication at scale


## Getting started
Before getting started with this guide, one needs to download and install the Graphlab Python library following the instructions on the Dato webpage: https://dato.com/learn/userguide/install.html


In [92]:
import graphlab as gl
#Following line checks the input dataset is at the path where the script can see it
!ls Amazon-GoogleProducts

Amazon.csv                              GoogleProducts.csv
Amzon_GoogleProducts_perfectMapping.csv


## Reading the data

After making sure that the core graphlab library is installed and the dataset is at place, proceed with reading the data into a graphlab's SFrame data structure. Graphlab provides the "batteries-included" functionaility for reading data:

In [93]:
amazon = gl.SFrame.read_csv("Amazon-GoogleProducts/Amazon.csv")

#print type(amazon)
print amazon.head()

PROGRESS: Finished parsing file /Users/asvyatko/Desktop/ProfMachanavajjhala/dataDeduplication/Amazon-GoogleProducts/Amazon.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.138927 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/asvyatko/Desktop/ProfMachanavajjhala/dataDeduplication/Amazon-GoogleProducts/Amazon.csv
PROGRESS: Parsing completed. Parsed 1363 lines in 0.068312 secs.
+------------+-------------------------------+-------------------------------+
|     id     |             title             |          description          |
+------------+-------------------------------+-------------------------------+
| b000jz4hqo | clickart 950 000 - premier

Working with SFrames is similar to that with pandas DataFrames. Check the column names for the SFrame and print out the top lines:  

In [94]:
#amazon and google have different title for the product name
amazon["name"] = amazon["title"]
amazon.head()
#print column names
amazon.column_names()
#modify column names to make in the same as Google's
amazon = amazon[['id', 'name','description', 'manufacturer', 'price']]
amazon.head()

id,name,description,manufacturer,price
b000jz4hqo,clickart 950 000 -premier image pack (dvd- ...,,broderbund,0.0
b0006zf55o,ca international -arcserve lap/desktop oem ...,oem arcserve backup v11.1win 30u for laptops and ...,computer associates,0.0
b00004tkvy,noah's ark activitycenter (jewel case ages ...,,victory multimedia,0.0
b000g80lqo,peachtree by sage premiumaccounting for nonpro ...,peachtree premiumaccounting for nonpro ...,sage software,599.99
b0006se5bq,singing coach unlimited,singing coach unlimited -electronic learning ...,carry-a-tune technologies,99.99
b000ehpzv8,emc retrospect 7.5 diskto disk windows ...,emc retrospect 7.5 diskto diskcromwindows ...,dantz,0.0
b00021xhzw,adobe after effectsprofessional 6.5 upgrade ...,upgrade only;installation of after ...,adobe,499.99
b000gzwjgc,acad upgrade dragonnaturallyspeaking pro ...,- marketing information:dragon naturallyspeak ...,nuance academic,399.54
b0000dbykm,mia's math adventure:just in time ...,in mia's math adventure:just in time children ...,kutoka,19.99
b00029bqa2,disney's 1st & 2nd gradebundle (pixar 1st grade ...,disney's 1st & 2nd gradebundle will help your ...,disney,14.99


Repeat the previous few steps for the GoogleProducts dataset, starting with loading data into SFrame:


In [95]:
google = gl.SFrame.read_csv("Amazon-GoogleProducts/GoogleProducts.csv")

PROGRESS: Finished parsing file /Users/asvyatko/Desktop/ProfMachanavajjhala/dataDeduplication/Amazon-GoogleProducts/GoogleProducts.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.045106 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/asvyatko/Desktop/ProfMachanavajjhala/dataDeduplication/Amazon-GoogleProducts/GoogleProducts.csv
PROGRESS: Parsing completed. Parsed 3226 lines in 0.054418 secs.


And printing the top lines and the column names (you will see that the name of the second column is now the same as Amazon's):


In [96]:
google.head()
google.column_names()

['id', 'name', 'description', 'manufacturer', 'price']

## Feature extraction and model selection

Once the data is loaded into SFrames, we can go ahead and select the features for classification and defining the distance measures. 

Graphlab provides a nearest_neighbors toolkit which labels a pair of records as duplicate if one record is a close neighbor of the other based on some distance measure. To resolve the question of transitive closure---A and B are duplicates, B and C are duplicates, but A and C are not---this model constructs a similarity graph and finds the connected components. Each connected component corresponds to an entity in the final output.

In the following cell, try out a few distance measures including the Jaccard, Euclidean, Manhattan and Levenstein (uncommenting one line starting with 'dist' or 'dist_spec' at a time) and rerunning the model training step:

In [99]:
dist = [[('name','description',), 'jaccard', 3]]
#dist = [[('price',), 'manhattan', 0.3]]
#dist = [[('manufacturer',), 'jaccard', 1]]
#dist_spec = [[('X1', 'X2'), 'euclidean', 2],
#              [('species',), 'levenshtein', 0.3]]
model = gl.nearest_neighbor_deduplication.create({'amazon':amazon,'google':google}, row_label='id', grouping_features=['manufacturer'], distance=dist)

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1            | 2       | 50          | 945us        |
PROGRESS: | Done         |         | 100         | 1.226ms      |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: +-----------------------------+
PROGRESS: | Number of components merged |
PROGRESS: +-----------------------------+
PROGRESS: | 1                           |
PROGRESS: | 0                           |
PROGRESS: +-----------------------------+
PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1            | 1       | 100         | 597us        |
PROGRESS

Understand and compare the similarity reports for each:
    

##Summarize model details

In [100]:
model.summary()

Class                               : NearestNeighborDeduplication

Schema
------
Number of input datasets            : 2
Number of feature columns           : 2
Number of neighbors per point (k)   : 2
Max distance to a neighbor (radius) : None
Number of entities                  : 1285
Total training time (seconds)       : 135.2382

Training
--------
Total training time (seconds)       : 135.2382

Accessible fields                   : 
   entities                         : Consolidated input records plus entity labels.


## Finding the duplicates

Having trained the model based on the graph kNN classifier with some distance measure, continue to find the duplicates in the two datasets.

The model's entities attribute contains the deduplication results. All input data rows are appended into a single SFrame, and the column **__entity** indicates which records correspond to the same entity.


In [101]:
m1=model
entities1 = m1['entities']

Basic analogs of SQL GROUP BY, SELECT, and HAVING COUNT are suppored on graphlab SFrames. Which will allow us to easily group entities by count, select those which have at least 1 duplicate and filter those from the initial frame:


In [102]:
# find the dupe IDs
import graphlab.aggregate as agg

entity_counts1 = m1['entities'].groupby('__entity', agg.COUNT)
dupe_entities1 = entity_counts1[entity_counts1['Count'] > 1]['__entity']
# print the results for the dupes
dupes1 = m1['entities'].filter_by(dupe_entities1, '__entity')
#dupes1.print_rows(40, max_row_width=100, max_column_width=50)
dupes1.column_names()

['__sframe', 'id', '__entity', 'name', 'description', 'manufacturer']

## Plugging Sframes into Pandas

Graphlab provides an interface to plug the Sframes into more existing packages like pandas. First of all, converting the Sframe to DataFrames:

In [103]:
dupes1pd = dupes1.to_dataframe()
print dupes1pd.head()

  __sframe                                                 id  __entity  \
0   google  http://www.google.com/base/feeds/snippets/1841...         0   
1   amazon                                         b0009eu0ca         0   
2   amazon                                         b00004wi4b         3   
3   amazon                                         b000gv8u32         3   
4   amazon                                         b000f7bps4        10   

                                            name  \
0  sony playstation gretzky nhl 2006 psp - 98627   
1                                sony acid pro 5   
2                                      startopia   
3                            tomb raider: legend   
4               palo alto marketing plan pro 9.0   

                                         description        manufacturer  
0  all-new wayne vs. wayne mode where gamers earn...                sony  
1  acid pro 5 puts total composition and editing ...                sony  
2  startopi

The usual Pandas equiavalents of SQL are now possible.  

In [104]:
grouped  = dupes1pd.groupby(['__entity',"__sframe"]).size()

Following resets the indexing in the dataframe, compare to the outpout of the  

In [125]:
import pandas as pd
nat = pd.DataFrame(grouped.reset_index())
print len(grouped)

1141


For convenience, rename the last column of the data frame to "counts":


In [106]:
nat.columns = ["__entity","__sframe", "counts"]
nat.head(20)

,__entity,__sframe,counts
0,0,amazon,1
1,0,google,1
2,3,amazon,2
3,10,amazon,3
4,13,amazon,8
5,17,amazon,3
6,17,google,1
7,18,amazon,3
8,18,google,1
9,26,amazon,2


In the following, we are going to be looking for exact duplicate matches between the Amazon and Google datasets. We can obtain those by selecting counts==1:


In [137]:
#dupes = nat[nat.counts == 1]
#dupes1[['__sframe','__entity', 'name', 'manufacturer']]
#print len(nat)
print dupes1

+----------+-------------------------------+----------+
| __sframe |               id              | __entity |
+----------+-------------------------------+----------+
|  google  | http://www.google.com/base... |    0     |
|  amazon  |           b0009eu0ca          |    0     |
|  amazon  |           b00004wi4b          |    3     |
|  amazon  |           b000gv8u32          |    3     |
|  amazon  |           b000f7bps4          |    10    |
|  amazon  |           b000gkshi6          |    10    |
|  amazon  |           b000gwfgqk          |    10    |
|  amazon  |           b0009gd0km          |    13    |
|  amazon  |           b000fqvxge          |    13    |
|  amazon  |           b000jbxxtk          |    13    |
+----------+-------------------------------+----------+
+-------------------------------+-------------------------------+
|              name             |          description          |
+-------------------------------+-------------------------------+
| sony playstation

## Duplicate detection accuracy

Let us first pick one of the elements of the duplicate dataset and inspect it, the top row has ID *b00004tkvy*:

In [108]:
dupes1.filter_by('b00004tkvy','id')

__sframe,id,__entity,name,description,manufacturer
amazon,b00004tkvy,1080,noah's ark activitycenter (jewel case ages ...,,victory multimedia


The corresponding entity is *1080*

In [109]:
dupes1.filter_by(1080,'__entity')['id']

dtype: str
Rows: 3
['b00004tkvy', 'b00004t2un', 'b000636jii']

In [110]:
google.filter_by('http://www.google.com/base/feeds/snippets/18434262004669557918', 'id')

id,name,description,manufacturer,price
http://www.google.com/base/feeds/snippets/1843 ...,webroot software inc -31250 - spy sweeper ...,a sypware infection is nolonger just a matter of ...,,24.81


To validate the result and to estimate the accuracy, we use the dataset of known (labeled) matches between the Google and Amazon datasets:


In [111]:
perfect = gl.SFrame.read_csv("Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv")

PROGRESS: Finished parsing file /Users/asvyatko/Desktop/ProfMachanavajjhala/dataDeduplication/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.088222 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/asvyatko/Desktop/ProfMachanavajjhala/dataDeduplication/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv
PROGRESS: Parsing completed. Parsed 1300 lines in 0.015416 secs.


As we can check below, the dataset contains 1300 rows of the amazon-google duplicate products:

In [112]:
#print perfect.column_names()
#print perfect
#perfect['idGoogleBase']
#perfect['idAmazon']
perfect['idAmazon' == 'b000jz4hqo']

{'idAmazon': 'b000jz4hqo',
 'idGoogleBase': 'http://www.google.com/base/feeds/snippets/18441480711193821750'}

However, the duplicate matches are not all one-to-one:


In [115]:
print "Number of unique Amazon elements: ", len(set(perfect["idAmazon"]))
print "Number of unique Google elements: ", len(set(perfect["idGoogleBase"]))
print "Total number of matches: ", len(perfect["idGoogleBase"])

Number of unique Amazon elements:  1113
Number of unique Google elements:  1291
Total number of matches:  1300


The number of duplicates identified in the real unlabeled dataset is:

In [162]:
#len(nat)
type (nat)
print nat

      __entity __sframe   0
0            0   amazon   1
1            0   google   1
2            3   amazon   2
3           10   amazon   3
4           13   amazon   8
5           17   amazon   3
6           17   google   1
7           18   amazon   3
8           18   google   1
9           26   amazon   2
10          28   amazon   3
11          31   amazon   3
12          32   amazon   2
13          35   amazon   4
14          35   google   1
15          36   amazon   2
16          37   amazon   2
17          38   amazon   2
18          39   amazon   3
19          40   amazon   3
20          41   amazon   4
21          43   amazon   3
22          45   amazon   5
23          46   amazon   2
24          47   amazon   4
25          48   amazon   5
26          50   amazon   2
27          51   amazon  10
28          52   amazon   2
29          53   amazon   3
...        ...      ...  ..
1111      1253   amazon   8
1112      1254   amazon   3
1113      1255   amazon   2
1114      1260   ama

Considering one-to-one matches, the number of duplicates identified is within 3% close to that from the perfect labeled dataset.

Next step is to evaluate the precision, recall and F-measure including one-to-many matches along with one-to-one.